In [1]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import geopandas as gpd, descartes
import itertools
import os
import pickle

# Estimate SNF/ALF locs

ACS data from: https://data.census.gov/cedsci/advanced

shapefiles from: https://www.census.gov/cgi-bin/geo/shapefiles/index.php

census block data from: https://ciser.cornell.edu/data/data-archive/census-2010-sf1-download-center/

## pull in census age/race counts per block

In [2]:
# define vars needed
locs = ['STATE','COUNTY','TRACT','BLKGRP','BLOCK']
all_pop = ['P0010001']
all_male = ['P01200' + str(i) for i in  list(range(20,26))]
all_fem = ['P01200' + str(i) for i in  list(range(44,50))]

# define race vars needed
white_male = ['P012A0' + str(i) for i in list(range(20,26))]
white_fem = ['P012A0' + str(i) for i in list(range(44,50))]

black_male = ['P012B0' + str(i) for i in list(range(20,26))]
black_fem = ['P012B0' + str(i) for i in list(range(44,50))]

amer_ind_male = ['P012C0' + str(i) for i in list(range(20,26))]
amer_ind_fem = ['P012C0' + str(i) for i in list(range(44,50))]

asian_male = ['P012D0' + str(i) for i in list(range(20,26))]
asian_fem = ['P012D0' + str(i) for i in list(range(44,50))]

hawaii_male = ['P012E0' + str(i) for i in list(range(20,26))]
hawaii_fem = ['P012E0' + str(i) for i in list(range(44,50))]

other_male = ['P012F0' + str(i) for i in list(range(20,26))]
other_fem = ['P012F0' + str(i) for i in list(range(44,50))]

mixed_male = ['P012G0' + str(i) for i in list(range(20,26))]
mixed_fem = ['P012G0' + str(i) for i in list(range(44,50))]

In [3]:
total_counts_races = ['P012' + i + '001' for i in ['A','B','C','D','E','F','G']]

In [4]:
race_var_names = [str(i) + str(j) for i in ['white_','black_','amer_ind_','asian_',
                                       'hawaii_','other_','mixed_']
             for j in ['male', 'fem']]
race_vars = []
for name in race_var_names:
    race_vars += locals()[name]

In [5]:
which_cols = locs + all_pop + all_fem + all_male + race_vars + total_counts_races

In [6]:
%%time
# read in decennial census

decennial = pd.read_csv('/home/j/temp/beatrixh/sim_science/decennial_census_2010/wa2010ur1_all_vars_csv/wa2010ur1_all_vars.CSV', usecols=which_cols)
decennial = decennial[decennial.BLOCK.notna()]

CPU times: user 22.3 s, sys: 1.5 s, total: 23.8 s
Wall time: 24.6 s


## create geoid key to merge with other datasets

In [8]:
# get a list of all cols needed for geoid
lower_locs = [i.lower() for i in locs]

# convert to strings
decennial[lower_locs] = decennial[locs].astype(float).astype(int).astype(str)
decennial[lower_locs] = decennial[lower_locs].astype(str)

# rearrange loc vars to be strings of uniform length
decennial.county = [('00' + i) if len(i)==1
             else '0' + i if len(i)==2
             else i for i in decennial.county]

decennial.tract = ['0' + i if len(i)==5
            else '00' + i if len(i)==4
            else '000' + i if len(i)==3
            else i for i in decennial.tract]

decennial.block = ['0' + i if len(i)==3
            else '00' + i if len(i)==2
            else '000' + i if len(i)==1
            else i for i in decennial.block]

# add geoid
decennial['geoid'] = decennial['state'] + decennial['county'] + decennial['tract'] + decennial['block']
print(decennial['geoid'].head())

#these should all be length 15:
print(set([len(i) for i in decennial.geoid]))

59    530019502003244
60    530019502003250
61    530019502003251
62    530019502003252
63    530019502003254
Name: geoid, dtype: object
{15}


In [43]:
decennial['tract_geoid'] = decennial.geoid.str[:-4]

## aggregate elder vars

In [9]:
# create lists holding all vars all races, split by sex.
fem_elder_names = [str(i) + 'fem' for i in ['white_','black_','amer_ind_','asian_',
                                       'hawaii_','other_','mixed_']]
male_elder_names = [str(i) + 'male' for i in ['white_','black_','amer_ind_','asian_',
                                        'hawaii_','other_','mixed_']]
fem_vars = []
for name in fem_elder_names:
    fem_vars += locals()[name]
    
male_vars = []
for name in male_elder_names:
    male_vars += locals()[name]

In [10]:
# rename cols
decennial = decennial.rename(columns={'P0010001' : 'total'})
decennial = decennial.rename(columns={'P012A001' : 'white_total',
                                      'P012B001' : 'black_total',
                                      'P012C001' : 'amer_ind_total',
                                      'P012D001' : 'asian_total',
                                      'P012E001' : 'hawaii_total',
                                      'P012F001' : 'other_total',
                                      'P012G001' : 'mixed_total',})

In [11]:
decennial['white_elder'] = decennial[white_male + white_fem].sum(axis=1)
decennial['black_elder'] = decennial[black_male + black_fem].sum(axis=1)
decennial['amer_ind_elder'] = decennial[amer_ind_male + amer_ind_fem].sum(axis=1)
decennial['asian_elder'] = decennial[asian_male + asian_fem].sum(axis=1)
decennial['hawaii_elder'] = decennial[hawaii_male + hawaii_fem].sum(axis=1)
decennial['other_elder'] = decennial[other_male + other_fem].sum(axis=1)
decennial['mixed_elder'] = decennial[mixed_male + mixed_fem].sum(axis=1)

In [12]:
decennial['elder_count'] = decennial[fem_vars + male_vars].sum(axis=1)
decennial['elder_prop'] = decennial['elder_count'] / decennial['total']

In [13]:
drop_cols = locs + all_fem + all_male + race_vars

In [14]:
decennial.drop(drop_cols, axis = 1) #you could take this? but consider keeping loc vars in

,total,white_total,black_total,amer_ind_total,asian_total,hawaii_total,other_total,mixed_total,state,county,...,geoid,white_elder,black_elder,amer_ind_elder,asian_elder,hawaii_elder,other_elder,mixed_elder,elder_count,elder_prop
59,0,0,0,0,0,0,0,0,53,001,...,530019502003244,0,0,0,0,0,0,0,0,NaN
60,0,0,0,0,0,0,0,0,53,001,...,530019502003250,0,0,0,0,0,0,0,0,NaN
61,0,0,0,0,0,0,0,0,53,001,...,530019502003251,0,0,0,0,0,0,0,0,NaN
62,0,0,0,0,0,0,0,0,53,001,...,530019502003252,0,0,0,0,0,0,0,0,NaN
63,0,0,0,0,0,0,0,0,53,001,...,530019502003254,0,0,0,0,0,0,0,0,NaN


## read in ACS disability vars

In [27]:
os.listdir('/home/j/temp/beatrixh/sim_science/ACS_2018/ACSST5Y2018.S1810_WA')

['ACSST5Y2018.S1810_data_with_overlays_2020-04-21T032059.csv',
 'ACSST5Y2018.S1810_metadata_2020-04-21T032059.csv',
 'ACSST5Y2018.S1810_table_title_2020-04-21T032059.txt']

In [28]:
disab_meta = pd.read_csv('/home/j/temp/beatrixh/sim_science/ACS_2018/ACSST5Y2018.S1810_WA/ACSST5Y2018.S1810_metadata_2020-04-21T032059.csv')
disab = pd.read_csv('/home/j/temp/beatrixh/sim_science/ACS_2018/ACSST5Y2018.S1810_WA/ACSST5Y2018.S1810_data_with_overlays_2020-04-21T032059.csv')

In [35]:
# create a key for census tracts
disab['tract_geoid'] = disab.GEO_ID.str[9:]

In [44]:
# grab disability vars to merge onto counts
elder_disab = disab_meta.loc[(disab_meta.id.str.contains('Estimate!!Percent with a disability!!Subject!!DISABILITY TYPE BY DETAILED AGE!!')) &
               (disab_meta.id.str.contains('!!Population 65 years and over')) &
                       (~disab_meta.id.str.contains('7'))].GEO_ID.values
list(elder_disab)

['S1810_C03_026E',
 'S1810_C03_036E',
 'S1810_C03_044E',
 'S1810_C03_052E',
 'S1810_C03_060E',
 'S1810_C03_067E']

In [171]:
decennial.head()

,STATE,COUNTY,TRACT,BLKGRP,BLOCK,total,P0120020,P0120021,P0120022,P0120023,...,white_elder,black_elder,amer_ind_elder,asian_elder,hawaii_elder,other_elder,mixed_elder,elder_count,elder_prop,tract_geoid
59,53,1.0,950200.0,3.0,3244.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,53001950200
60,53,1.0,950200.0,3.0,3250.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,53001950200
61,53,1.0,950200.0,3.0,3251.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,53001950200
62,53,1.0,950200.0,3.0,3252.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,53001950200
63,53,1.0,950200.0,3.0,3254.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,NaN,53001950200


In [52]:
# merge ACS onto decennial counts
counts = decennial.merge(disab.loc[1:,list(elder_disab) + ['tract_geoid']], on='tract_geoid', how='right')
counts = counts.rename(columns = {'S1810_C03_026E':'pct_hearing_disab',
                                  'S1810_C03_036E':'pct_vision_disab',
                                  'S1810_C03_044E':'pct_cog_disab',
                                  'S1810_C03_052E':'pct_amb_disab',
                                  'S1810_C03_060E':'pct_selfcare_disab',
                                  'S1810_C03_067E':'pct_ind_living_disab'})

## find blocks where elder prop higher than in surrounding area

In [58]:
def save_obj(obj, filename):
    with open(filename + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename + '.pkl', 'rb') as f:
        return pickle.load(f)

In [121]:
block_nbs = load_obj('2block_nbs_WA_2020_04_18')

In [136]:
%%time

counts['lmax'] = 0 #initialize local maxima
counts['surrounding_elder_prop'] = 1

for i in range(counts.shape[0]): #for each blockgroup
    neighbors = block_nbs[counts.iloc[i].geoid] #get list of adjacent blocks
    denom = counts[counts.geoid.isin(neighbors)].total.sum() #total pop of surrounding blocks
    num = counts[counts.geoid.isin(neighbors)].elder_count.sum() #total 65+ pop of surrounding blocks
    counts.iloc[i,-1] = (num/denom) #set surrounding_elder_prop
    if counts.iloc[i].elder_prop > (num/denom):
        counts.iloc[i,-2] = 1 #set lmax

/share/code/beatrixh/miniconda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in long_scalars
  
/share/code/beatrixh/miniconda/envs/gis/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


CPU times: user 56min 5s, sys: 10.3 s, total: 56min 15s
Wall time: 56min 23s


In [ ]:
# %%time

# counts['lmax_broader'] = 0 #initialize local maxima
# counts['broad_surr_elder_prop'] = 1

# for i in range(counts.shape[0]): #for each blockgroup
#     neighbors = block_nbs[counts.iloc[i].geoid] #get list of adjacent blocks
#     nbs_nbs = list(itertools.chain(*[blkgrp_nbs[i] for i in neighbors])) #get neighbors of neighbors
#     nbs_nbs = [geoid for geoid in nbs_nbs if counts.iloc[i].geoid != geoid] #get rid of self
#     denom = counts[counts.geoid.isin(nbs_nbs)].total.sum() #total pop of surrounding blocks
#     num = counts[counts.geoid.isin(nbs_nbs)].elder_count.sum() #total 65+ pop of surrounding blocks
#     counts.iloc[i,-1] = (num/denom) #set broad_surr_elder_prop
#     if counts.iloc[i].elder_prop > (num/denom): #if self has greater prop than surrounding area,
#         counts.iloc[i,-2] = 1 #set lmax_broader

In [139]:
counts['prop_ratio'] = counts['elder_prop'] / counts['surrounding_elder_prop']
counts['prop_diff'] = counts['elder_prop'] - counts['surrounding_elder_prop']

,STATE,COUNTY,TRACT,BLKGRP,BLOCK,total,P0120020,P0120021,P0120022,P0120023,...,elder_prop,tract_geoid,pct_hearing_disab,pct_vision_disab,pct_cog_disab,pct_amb_disab,pct_selfcare_disab,pct_ind_living_disab,lmax,surrounding_elder_prop
6,53,1.0,950200.0,3.0,3256.0,28,0,0,0,0,...,0.000000,53001950200,16.3,3.0,4.5,16.9,2.7,3.9,0,0.083333
7,53,1.0,950200.0,3.0,3257.0,9,0,0,0,0,...,0.000000,53001950200,16.3,3.0,4.5,16.9,2.7,3.9,0,0.058824
10,53,1.0,950200.0,3.0,3261.0,4,0,0,0,0,...,0.000000,53001950200,16.3,3.0,4.5,16.9,2.7,3.9,0,0.044944
11,53,1.0,950200.0,3.0,3262.0,33,0,0,0,0,...,0.060606,53001950200,16.3,3.0,4.5,16.9,2.7,3.9,1,0.052632
12,53,1.0,950200.0,3.0,3263.0,5,0,0,0,0,...,0.200000,53001950200,16.3,3.0,4.5,16.9,2.7,3.9,1,0.111111


## plots

In [158]:
wa_block_shapes = gpd.read_file('/home/j/temp/beatrixh/sim_science/census_GIS/tl_2010_53_tabblock10/tl_2010_53_tabblock10.shp')

In [159]:
wa_block_shapes = wa_block_shapes.rename(columns={'GEOID10': 'GEOID','COUNTYFP10':'COUNTY'})

## pull in snf counts

In [101]:
beds = pd.read_csv('/home/j/temp/beatrixh/sim_science/snfs/WA_snfs_2010.csv')

In [102]:
# turn into geodataframe for plotting
beds = gpd.GeoDataFrame(beds, geometry=gpd.points_from_xy(beds.best_long,beds.best_lat))

In [141]:
beds.head()

,name,state,county,address,best_lat,best_long,geometry
0,AVALON CARE CENTER - OTHELLO LLC,WA,ADAMS,"495 NORTH 13TH STREET, OTHELLO, WA",46.831165,-119.156924,POINT (-119.15692 46.83117)
1,EAST ADAMS CARE CENTER LLC,WA,ADAMS,"506 S JACKSON STREET, RITZVILLE, WA",47.121304,-118.376988,POINT (-118.37699 47.12130)
2,PRESTIGE CARE & REHAB - CLARKSTON,WA,ASOTIN,"1242 11TH ST, CLARKSTON, WA",46.406043,-117.052202,POINT (-117.05220 46.40604)
3,LIFE CARE CENTER OF KENNEWICK,WA,BENTON,"1508 WEST SEVENTH AVENUE, KENNEWICK, WA",46.202530,-119.139507,POINT (-119.13951 46.20253)
4,REGENCY KENNEWICK REHAB & NSG CENTER,WA,BENTON,"2702 S ELY STREET, KENNEWICK, WA",46.183745,-119.159171,POINT (-119.15917 46.18375)


In [143]:
# add col to wa_block_shapes for each block with an snf

In [189]:
%%time

wa_block_shapes['has_snf'] = 0

for index, snf in beds.iterrows():
    block_geoid = wa_block_shapes[~wa_block_shapes.geometry.disjoint(snf.geometry)].GEOID.tolist()
    wa_block_shapes.loc[wa_block_shapes.GEOID.isin(block_geoid),'has_snf'] = 1

CPU times: user 2min 27s, sys: 5.3 s, total: 2min 33s
Wall time: 2min 33s


In [190]:
wa_block_shapes.head()

,STATEFP10,COUNTY,TRACTCE10,BLOCKCE10,GEOID,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,has_snf
0,53,001,950100,1004,530019501001004,Block 1004,G5040,R,None,None,S,0,477164,+47.2397419,-117.9780911,"POLYGON ((-117.98390 47.23402, -117.98440 47.2...",0
1,53,001,950100,1220,530019501001220,Block 1220,G5040,R,None,None,S,950028,0,+47.2132824,-117.9639088,"POLYGON ((-117.96049 47.22108, -117.96047 47.2...",0
2,53,001,950100,1226,530019501001226,Block 1226,G5040,R,None,None,S,606144,0,+47.1896716,-117.9620025,"POLYGON ((-117.96031 47.19978, -117.96026 47.1...",0
3,53,001,950100,1235,530019501001235,Block 1235,G5040,R,None,None,S,7599,0,+47.0853664,-117.9938390,"POLYGON ((-117.99392 47.08451, -117.99393 47.0...",0
4,53,001,950100,1228,530019501001228,Block 1228,G5040,R,None,None,S,3643139,0,+47.1079086,-117.9687545,"POLYGON ((-117.95994 47.09514, -117.96034 47.0...",0


In [192]:
wa_block_shapes.has_snf.sum()

219

In [196]:
wa_block_shapes.columns

Index(['STATEFP10', 'COUNTY', 'TRACTCE10', 'BLOCKCE10', 'GEOID', 'NAME10',
       'MTFCC10', 'UR10', 'UACE10', 'UATYP10', 'FUNCSTAT10', 'ALAND10',
       'AWATER10', 'INTPTLAT10', 'INTPTLON10', 'geometry', 'has_snf'],
      dtype='object')

In [198]:
wa_block_shapes.GEOID

0         530019501001004
1         530019501001220
2         530019501001226
3         530019501001235
4         530019501001228
               ...       
195569    530770028023014
195570    530770014001022
195571    530770030023027
195572    530770030011003
195573    530770030012087
Name: GEOID, Length: 195574, dtype: object

In [204]:
counts = counts.merge(wa_block_shapes[['GEOID','has_snf']], left_on = 'geoid', right_on='GEOID')

In [205]:
counts.head()

,STATE,COUNTY,TRACT,BLKGRP,BLOCK,total,P0120020,P0120021,P0120022,P0120023,...,pct_hearing_disab,pct_vision_disab,pct_cog_disab,pct_amb_disab,pct_selfcare_disab,pct_ind_living_disab,lmax,surrounding_elder_prop,GEOID,has_snf
0,53,1.0,950200.0,3.0,3244.0,0,0,0,0,0,...,16.3,3.0,4.5,16.9,2.7,3.9,0,NaN,530019502003244,0
1,53,1.0,950200.0,3.0,3250.0,0,0,0,0,0,...,16.3,3.0,4.5,16.9,2.7,3.9,0,0.060606,530019502003250,0
2,53,1.0,950200.0,3.0,3251.0,0,0,0,0,0,...,16.3,3.0,4.5,16.9,2.7,3.9,0,0.028571,530019502003251,0
3,53,1.0,950200.0,3.0,3252.0,0,0,0,0,0,...,16.3,3.0,4.5,16.9,2.7,3.9,0,NaN,530019502003252,0
4,53,1.0,950200.0,3.0,3254.0,0,0,0,0,0,...,16.3,3.0,4.5,16.9,2.7,3.9,0,NaN,530019502003254,0


In [206]:
counts[(counts.total==0) & (counts.has_snf==1)]

,STATE,COUNTY,TRACT,BLKGRP,BLOCK,total,P0120020,P0120021,P0120022,P0120023,...,pct_hearing_disab,pct_vision_disab,pct_cog_disab,pct_amb_disab,pct_selfcare_disab,pct_ind_living_disab,lmax,surrounding_elder_prop,GEOID,has_snf
14634,53,9.0,1200.0,1.0,1070.0,0,0,0,0,0,...,16.0,5.7,6.3,16.9,2.0,10.3,0,0.568075,530090012001070,1
65245,53,33.0,6300.0,2.0,2002.0,0,0,0,0,0,...,6.6,3.0,5.0,12.0,7.3,10.9,0,0.575368,530330063002002,1
67041,53,33.0,8600.0,2.0,2007.0,0,0,0,0,0,...,6.1,24.7,24.7,31.9,18.6,25.8,0,0.010681,530330086002007,1
122142,53,53.0,73406.0,3.0,3006.0,0,0,0,0,0,...,19.3,0.0,5.0,38.0,7.5,8.1,0,0.088850,530530734063006,1
128570,53,53.0,61602.0,1.0,1033.0,0,0,0,0,0,...,0.0,21.4,17.9,32.1,14.3,17.9,0,0.000000,530530616021033,1


In [210]:
counts.to_csv('/home/j/temp/beatrixh/sim_science/outputs/WA_snf_dt_with_independent_vars.csv')